In [1]:
import csv
import json
import sys
import pandas as pd
from pathlib import Path
import requests
import pprint
sys.path.insert(0, 'C:/Users/sam/Desktop/ScrapeTest') 
from cookie_monster import COOKIE
import demjson
pd.set_option("display.max_rows", None, "display.max_columns", None)

user_cookie = dict()

for item in COOKIE.split("; "):
    k, v = item.split("=")
    user_cookie[k] = v

urllist = ["https://lpl.qq.com/es/stats.shtml?bmid=7325",
        "https://lpl.qq.com/es/stats.shtml?bmid=6909"]

test_url = "https://lpl.qq.com/es/stats.shtml?bmid=6909"

#This is pulling the champion data.
champ_dd = requests.get("https://ddragon.leagueoflegends.com/cdn/11.8.1/data/en_US/champion.json", cookies=user_cookie)
champ_json = champ_dd.json()
champ_name = champ_json["data"] #this creates a list of all the champions
champ_key_dict = {}
for i in champ_name:
    champ_key_dict.update({champ_json["data"][i]["key"]:i})
#pprint.pprint(champ_dd.json())
#control = {'side':'left', 'count':0, 'records':0}
#json_data['bMatchId']
#jd = json.loads(json_data["sMatchInfo"][0]['battleInfo']['BattleData'])
#jd['left']['players']

def lpl_url_for_request_scraping(raw_url, test):
    base_url = "https://lpl.qq.com/web201612/data/LOL_MATCH_DETAIL_"
    js = ".js"
    match_num = raw_url[-4:]
    new_url = base_url + match_num + js
    if test == True:
        pprint.pprint(new_url)
    return (new_url)

#Get the lpl json data. 
def lpl_get_match_data(url, test):
    good_url = lpl_url_for_request_scraping(url, False)
    json_file = requests.get(good_url)
    json_content = json.loads(json_file.text[12:-1]) # or demjson.decode(json_file.content[12:-1])
    #assert json_file.status_code == 200
    if test == True:
        pprint.pprint(json_content)
    return(json_content)

def lpl_build_dataframe(input_match_data, test):
    jd = json.loads(input_match_data["sMatchInfo"][0]['battleInfo']['BattleData'])
    #'bMatchId' is what matches the URL
    side = 'left'
    count = 0
    records = 0
    while count < 1:
        player_info = jd[side]['players'][count]
        count = count + 1
    return (player_info)


y = lpl_get_match_data(test_url, False)
z = lpl_build_dataframe(y, False)

print(z)

{'kill': '0', 'death': '0', 'assist': '2', 'gold': '10711', 'lasthit': '276', 'hero': '43', 'name': 'TES369', 'memberId': '2147', 'skill-1': '4', 'skill-2': '12', 'equip': {'game-equip-1': '6655', 'game-equip-2': '2055', 'game-equip-3': '3040', 'game-equip-4': '1082', 'game-equip-5': '6616', 'game-equip-6': '3158', 'game-equip-7': '3363'}, 'win': '0', 'totalDamage': '152963', 'totalDamageToChamp': '18444', 'pDamageToChamp': '1151', 'mDamageDealtToChamp': '17292', 'pDamageDealt': '26467', 'mDamageDealt': '122465', 'largestCriticalStrike': '0', 'totalHeal': '4249', 'totalDamageTaken': '11374', 'pDamageTaken': '5898', 'mDamageTaken': '5332', 'neutralKilled': '4', 'neutralKilledTJungle': '4', 'neutralKilledEJungle': '0', 'wardsPlaced': '13', 'wardsKilled': '10', 'visionWardsBought': '3', 'dKills': '0', 'tKills': '0', 'qKills': '0', 'pKills': '0', 'towerKills': '0', 'inhibitorKills': '0', 'deadTime': '0', 'level': '16', 'killingSprees': '0', 'largestMultiKill': '0', 'largestKillingSpree': '

Using legacy setup.py install for demjson, since package 'wheel' is not installed.
    Running setup.py install for demjson: started
    Running setup.py install for demjson: finished with status 'done'
Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'C:\Python38\python.exe -m pip install --upgrade pip' command.


In [ ]:
def get_data_lpl(json_content, test):
    list_list = []
    jd = json.loads(json_content["sMatchInfo"][0]['battleInfo']['BattleData']) #this returns a json
    keys = jd.keys()
    control = {'side':'left', 'count':0, 'records':0}
    gameId= json_content['bMatchId']
    #this subsection is to move between the 'right' and 'left' team lists because that's how this info is nested.
    while control['records'] < 10: #set this to 1 to get it to run for only one player
        output_list = []
        if control['count'] == 5:
            control['side'] = 'right'
            control['count'] = 0
        else: #I think the champion id -> champion modifier goes in here.
            side = control['side']
            count = control['count']
            data = jd[side]['players'][count] #dicts
        for i in a_keys:
            try:
                if i == 'hero': #this statement worked
                    output_list.append(str(champ_key_dict[jd[side]['players'][count][i]]))
                elif data[i]:
                    #print('yes')
                    output_list.append(data[i]) #list of all of the player info, which is currently fucking up
                elif i == 'firstBlood': #have to have this statement in here because apparently if the player didn't FB they just leave the line empty and python picks it up as False
                        if not data[i]:
                            data[i] = '0'
                            output_list.append(data[i])
                        else:
                            data[i] = '1'
                            output_list.append(data[i])
                elif i == 'game-id':
                    output_list.append(gameId)
                elif i == 'side':
                    output_list.append(control['side'])
                else:
                    print (i, 'not in lpl headers')
            except Exception as e: # probably dont need this full line because I'm doing something with the exception
                if i in b_keys:
                    print (i, "in non-lpl headers")
                    pass
                elif i == 'game-id' or 'side':
                    continue
                else:
                    print ("exception", i)
        list_list.append(output_list)
        control['count'] = control['count'] + 1
        control['records'] = control['records'] + 1
        print('done')
    if test == True:
        pprint.pprint(list_list)
    return(list_list)